In [1]:
import sys
sys.path.append('./build')

In [2]:
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, PointCloud

In [3]:
import numpy as np
np.set_printoptions(suppress=True)

In [4]:
import ppf_registration_spatial_hashing as ppf

In [5]:
model = ppf.load_ply_file('./samples/data/parasaurolophus_low_normals2.ply')

In [6]:
scene = ppf.load_ply_file('./samples/data/rs1_normals.ply')

In [11]:
def run(model, scene, tau_d, ref_point_df=5, scene_leaf_size=7, vote_count_threshold=0.3):
    minpt, maxpt = ppf.min_max_3d(model)
    model_d_dist = np.max(np.array([maxpt.x - minpt.x,
                                    maxpt.y - minpt.y, 
                                    maxpt.z - minpt.z])) * tau_d
    downsampled_model = ppf.voxelgrid_downsample(model, model_d_dist)
    downsampled_scene = ppf.voxelgrid_downsample(scene, scene_leaf_size)
    solution = ppf.ppf_registration(downsampled_scene,
                                    downsampled_model, 
                                    model_d_dist, 
                                    ref_point_df, 
                                    vote_count_threshold, cpu_clustering=False)
    return solution

In [24]:
T = run(model, scene, 0.05)
T

array([[   0.99961275,   -0.01596108,    0.0227909 , -101.37711   ],
       [   0.02769098,    0.6507142 ,   -0.7588176 , -558.8262    ],
       [  -0.0027188 ,    0.75915486,    0.65090436, -237.27734   ],
       [   0.        ,    0.        ,    0.        ,    1.        ]],
      dtype=float32)

In [25]:
transformed_model_points = (T @ np.vstack((model.array()[:3,:], np.ones(model.width))))[:3,:].T.reshape(-1,3)
transformed_model_points

array([[-163.77668715,  -48.32428516, -693.88526105],
       [-163.92386582,  -49.40905647, -694.34889883],
       [-163.13664221,  -48.74353932, -693.6697181 ],
       ...,
       [-163.3675347 , -111.14019672, -608.89912762],
       [-164.23510823, -111.19902517, -609.03410223],
       [-163.15136814, -110.51163782, -608.38903687]])

In [26]:
viewer = Viewer()
viewer.add(Grid())
modelvis = PointCloud(transformed_model_points * 0.01)
modelvis.position = [0,0,7]
viewer.add(modelvis)
scenevis = PointCloud(scene.array()[:3,:].T * 0.01, color='gray')
scenevis.position = [0,0,7]
viewer.add(scenevis)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…